# RAG 시스템에서의 Reranking 결정 방법

## 개요
Reranking은 검색-증강 생성(RAG) 시스템에서 중요한 단계로, 검색된 문서의 관련성과 품질을 개선하는 것을 목표로 합니다. 그것은 초기에 검색된 문서를 재평가하고 재정렬하여 후속 처리 또는 제시를 위해 가장 적절한 정보를 우선시하는 것을 포함합니다.

## 필요성
RAG 시스템의 경우, Reranking의 주요 동기는 초기 검색 방법의 한계를 극복하는 것으로, 이는 종종 더 단순한 유사성 메트릭에 의존합니다. Reranking은 기존 검색 기술에서 놓칠 수 있는 질의와 문서 간의 미묘한 관계를 고려하여 보다 정교한 관련성 평가를 가능하게 합니다. 이 프로세스는 생성 단계에서 가장 적절한 정보가 사용되도록 하여 RAG 시스템의 전반적인 성능을 향상시키는 것을 목표로 합니다.

## 주요 구성 요소
Reranking 시스템에는 일반적으로 다음 구성 요소가 포함됩니다.

1. 초기 검색기: 종종 임베딩 기반 유사도 검색을 사용하는 벡터 저장소입니다.
2. Reranking 모델: 이것은 둘 중 하나일 수 있습니다.
   - 관련성 평가를 위해 특별히 훈련된 대형 언어 모델(LLM)
   - 관련성 평가용으로 특별히 훈련된 교차 인코더 모델
3. 점수 매기기 메커니즘: 문서에 관련성 점수를 할당하기 위한 방법입니다.
4. 정렬 및 선택 논리: 새 점수에 따라 문서를 다시 정렬합니다.

## 방법 세부 정보
Reranking 프로세스는 일반적으로 이러한 단계를 따릅니다.

1. 초기 검색: 잠재적으로 관련된 초기 세트의 문서를 가져옵니다.
2. 페어 생성: 각 검색된 문서에 대해 쿼리-문서 쌍을 만듭니다.
3. 채점:
   - LLM 방법: 프롬프트를 사용하여 LLM에게 문서 관련 등급을 묻습니다.
   - 교차 인코더 방법: 쿼리-문서 쌍을 직접 모델에 공급합니다.
4. 점수 해석: 관련성 점수를 파싱하고 정규화합니다.
5. 순서 변경: 새로운 관련성 점수에 따라 문서를 정렬합니다.
6. 선택: 재정렬 목록에서 상위 K개의 문서를 선택합니다.

## 이 접근법의 이점
Reranking은 여러 가지 장점을 제공합니다.

1. 관련성 개선: 보다 정교한 모델을 사용하여 Reranking은 미묘한 관련성 요소를 포착할 수 있습니다.
2. 유연성: 특정 요구 사항 및 리소스에 따라 다른 Reranking 방법을 적용할 수 있습니다.
3. 향상된 컨텍스트 품질: RAG 시스템에 보다 관련성이 높은 문서를 제공함으로써 생성된 응답의 품질이 향상됩니다.
4. 노이즈 감소: Reranking은 덜 관련성이 있는 정보를 필터링하는 데 도움이 되며 가장 관련성이 높은 콘텐츠에 초점을 맞춥니다.

## 결론
Reranking은 RAG 시스템에서 강력한 기술로 검색된 정보의 품질을 크게 향상시킵니다. LLM 기반 점수 매기기 또는 특수화된 크로스 인코더 모델을 사용하는지 여부에 관계없이, Reranking은 문서 관련성의 미묘한 차이를 허용합니다. 이러한 개선된 관련성은 다운스트림 작업의 성능 향상으로 직결되므로, Reranking은 고급 RAG 구현에서 필수적인 구성 요소가 됩니다.

LLM 기반과 교차 인코더 Reranking 방법 사이의 선택은 필요한 정확도, 사용 가능한 계산 리소스 및 특정 애플리케이션 요구 사항과 같은 요인에 따라 달라집니다. 두 가지 접근 방식 모두 기본 검색 방법보다 실질적인 개선을 제공하며 RAG 시스템의 전반적인 효과성에 기여합니다.

### Import relevant libraries

In [54]:
import os
import sys
#from dotenv import load_dotenv
from langchain.docstore.document import Document
from typing import List, Dict, Any, Tuple
#from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.retrievers import BaseRetriever
from sentence_transformers import CrossEncoder


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
#from helper_functions import *
#from evaluation.evalute_rag import *

# Load environment variables from a .env file
#load_dotenv()

# Set the OpenAI API key environment variable
#os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Define the document's path

In [5]:
path = "../data/Understanding_Climate_Change.pdf"

### Create a vector store

In [65]:
vectorstore = encode_pdf(path)

## Method 1: LLM based function to rerank the retrieved documents

<div style="text-align: center;">

<img src="../images/rerank_llm.svg" alt="rerank llm" style="width:40%; height:auto;">
</div>

### Create a custom reranking function


In [67]:
class RatingScore(BaseModel):
    relevance_score: float = Field(..., description="The relevance score of a document to a query.")

def rerank_documents(query: str, docs: List[Document], top_n: int = 3) -> List[Document]:
    prompt_template = PromptTemplate(
        input_variables=["query", "doc"],
        template="""On a scale of 1-10, rate the relevance of the following document to the query. Consider the specific context and intent of the query, not just keyword matches.
        Query: {query}
        Document: {doc}
        Relevance Score:"""
    )
        
    llm_chain = prompt_template | llm.with_structured_output(RatingScore)
        
    scored_docs = []
    for doc in docs:
        input_data = {"query": query, "doc": doc.page_content}
        score = llm_chain.invoke(input_data).relevance_score
        try:
            score = float(score)
        except ValueError:
            score = 0  # Default score if parsing fails
        scored_docs.append((doc, score))
    
    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked_docs[:top_n]]

### Example usage of the reranking function with a sample query relevant to the document


In [69]:
query = "What are the impacts of climate change on biodiversity?"
initial_docs = vectorstore.similarity_search(query, k=15)
reranked_docs = rerank_documents(query, initial_docs)

# print first 3 initial documents
print("Top initial documents:")
for i, doc in enumerate(initial_docs[:3]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document


# Print results
print(f"Query: {query}\n")
print("Top reranked documents:")
for i, doc in enumerate(reranked_docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document

NotImplementedError: 

### Create a custom retriever based on our reranker

In [114]:
# Create a custom retriever class
class CustomRetriever(BaseRetriever, BaseModel):
    
    vectorstore: Any = Field(description="Vector store for initial retrieval")

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str, num_docs=2) -> List[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=30)
        return rerank_documents(query, initial_docs, top_n=num_docs)


# Create the custom retriever
custom_retriever = CustomRetriever(vectorstore=vectorstore)

# Create an LLM for answering questions
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Create the RetrievalQA chain with the custom retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True
)


### Example query


In [ ]:
result = qa_chain({"query": query})

print(f"\nQuestion: {query}")
print(f"Answer: {result['result']}")
print("\nRelevant source documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document

### Example that demonstrates why we should use reranking 

In [123]:
chunks = [
    "The capital of France is great.",
    "The capital of France is huge.",
    "The capital of France is beautiful.",
    """Have you ever visited Paris? It is a beautiful city where you can eat delicious food and see the Eiffel Tower. 
    I really enjoyed all the cities in france, but its capital with the Eiffel Tower is my favorite city.""", 
    "I really enjoyed my trip to Paris, France. The city is beautiful and the food is delicious. I would love to visit again. Such a great capital city."
]
docs = [Document(page_content=sentence) for sentence in chunks]


def compare_rag_techniques(query: str, docs: List[Document] = docs) -> None:
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(docs, embeddings)

    print("Comparison of Retrieval Techniques")
    print("==================================")
    print(f"Query: {query}\n")
    
    print("Baseline Retrieval Result:")
    baseline_docs = vectorstore.similarity_search(query, k=2)
    for i, doc in enumerate(baseline_docs):
        print(f"\nDocument {i+1}:")
        print(doc.page_content)

    print("\nAdvanced Retrieval Result:")
    custom_retriever = CustomRetriever(vectorstore=vectorstore)
    advanced_docs = custom_retriever.get_relevant_documents(query)
    for i, doc in enumerate(advanced_docs):
        print(f"\nDocument {i+1}:")
        print(doc.page_content)


query = "what is the capital of france?"
compare_rag_techniques(query, docs)

Comparison of Retrieval Techniques
Query: what is the capital of france?

Baseline Retrieval Result:

Document 1:
The capital of France is great.

Document 2:
The capital of France is beautiful.

Advanced Retrieval Result:

Document 1:
I really enjoyed my trip to Paris, France. The city is beautiful and the food is delicious. I would love to visit again. Such a great capital city.

Document 2:
Have you ever visited Paris? It is a beautiful city where you can eat delicious food and see the Eiffel Tower. 
    I really enjoyed all the cities in france, but its capital with the Eiffel Tower is my favorite city.


## Method 2: Cross Encoder models

<div style="text-align: center;">

<img src="../images/rerank_cross_encoder.svg" alt="rerank cross encoder" style="width:40%; height:auto;">
</div>

### Define the cross encoder class

In [6]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

class CrossEncoderRetriever(BaseRetriever, BaseModel):
    vectorstore: Any = Field(description="Vector store for initial retrieval")
    cross_encoder: Any = Field(description="Cross-encoder model for reranking")
    k: int = Field(default=5, description="Number of documents to retrieve initially")
    rerank_top_k: int = Field(default=3, description="Number of documents to return after reranking")

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str) -> List[Document]:
        # Initial retrieval
        initial_docs = self.vectorstore.similarity_search(query, k=self.k)
        
        # Prepare pairs for cross-encoder
        pairs = [[query, doc.page_content] for doc in initial_docs]
        
        # Get cross-encoder scores
        scores = self.cross_encoder.predict(pairs)
        
        # Sort documents by score
        scored_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)
        
        # Return top reranked documents
        return [doc for doc, _ in scored_docs[:self.rerank_top_k]]

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError("Async retrieval not implemented")



### Create an instance and showcase over an example

In [ ]:
# Create the cross-encoder retriever
cross_encoder_retriever = CrossEncoderRetriever(
    vectorstore=vectorstore,
    cross_encoder=cross_encoder,
    k=10,  # Retrieve 10 documents initially
    rerank_top_k=5  # Return top 5 after reranking
)

# Set up the LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Create the RetrievalQA chain with the cross-encoder retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=cross_encoder_retriever,
    return_source_documents=True
)

# Example query
query = "What are the impacts of climate change on biodiversity?"
result = qa_chain({"query": query})

print(f"\nQuestion: {query}")
print(f"Answer: {result['result']}")
print("\nRelevant source documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document